# <font color='#B31B1B'> Scikit-Learn </font>
We've already seen scikit-learn in action in class as a tool for fitting different machine learning models. But scikit-learn has even more to offer! In this section we'll review some basic syntax for fitting models in scikit-learn and explore tools for data pre-processing and hyperparameter tuning.

In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import warnings
warnings.simplefilter("ignore")

### <font color='#B31B1B'> Toy Data </font>

We'll use a custom 'toy' dataset that contains a nice mix of different data types.

In [3]:
data = pd.read_csv("data/data_processing.csv")

In [4]:
data.head()

,missing_col,col2,col3,outliers_col,outliers_col2,categorical_col,ordinal_col,text_col
0,19.0,21.0,1.579757,76,0.146815,mouse,normal,"The tunnel wound on and on, going fairly but n..."
1,12.0,87.0,0.257069,85,0.210569,dog,bad,"It had a perfectly round door like a porthole,..."
2,76.0,29.0,0.564826,49,2.427333,elephant,very bad,"It had a perfectly round door like a porthole,..."
3,42.0,66.0,0.345060,67,1.601458,elephant,very good,"It had a perfectly round door like a porthole,..."
4,49.0,43.0,0.861007,-14,22.375265,cat,very bad,"This hobbit was a very well-to-do hobbit, and ..."


Our goal is going to be to predict col3, which is a column with real valued features. This makes it **what kind of machine learning problem?**

### <font color='#B31B1B'> Pre-Processing Data </font>
For models like linear regression, we need all of our features to be numeric and not have any missing values. Luckily we've already seen some methods in class and homework to convert non-numeric features to a nice form. In scikit-learn we can connect all the steps to pre-process data into one operation called a pipeline.

For example, a possible pipeline for our synthetic data could be:

![pipeline](images/pipeline.png)

We are going to build this pipeline using scikit learn's [pipelines](http://scikit-learn.org/stable/modules/pipeline.html).

Scikit-learn [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) allows us to plug in together any number of transformers (that transform data) and estimators (that predict).

In [5]:
#Let's grab our different data types
target_variable = "col3"
numerical_cols =  data.drop(columns=target_variable).select_dtypes(np.number).columns
categorical_col = ['categorical_col']
ordinal_col = ["ordinal_col"]
text_col = ['text_col']

### <font color='#B31B1B'> Numerical Pre-processing </font>

We will create a pipeline for each kind of data first, then we will see how to put them together into a single Pipeline

First we can start by pre-processing our numeric data. Some steps we might want to include

- **Imputation**: Filling in missing values 

- **Scaling**: Normalizing our values (some options are max-min scaling, or subtracting mean and dividing by variance/standard deviation)

We can access both of these operations in the scikit-learn impute and preprocessing modules.

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy="mean") #Can set other strategies (check the help doc!)
scaler = StandardScaler()

If we wanted to run these alone (not in a pipeline), the general syntax is 'fit' (fitting parameters for transformation, like mean/median/mode) then 'transform' (applying transform with fitted parameters). Let's take a look at applying the imputer to the 'missing_col'.

In [7]:
#Before imputation
data[['missing_col']].isna().sum()

missing_col    96
dtype: int64

In [8]:
#Using imputation

imputed_col = imputer.fit(data[['missing_col']]).transform(data[['missing_col']])
#Or perform fit/transform together
imputed_col = imputer.fit_transform(data[['missing_col']])

pd.Series(imputed_col.reshape(-1)).isna().sum()

0

Where scikit-learn really shines is putting these operations together into a pipeline!

In [9]:
from sklearn.pipeline import make_pipeline

A sklearn Pipeline is defined as a sequence of steps.

For example, for the numerical variables our pipeline will have 2 steps:
- first we will impute,
- then we will scale. 

We can create this numerical pipeline like this:

In [10]:
numerical_pipeline = make_pipeline(
    imputer,
    scaler
)

In [11]:
numerical_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

Now we can use `fit` and `transform` with the pipeline and it will apply all the transformers sequentially.

In [12]:
numerical_pipeline.fit_transform(data[numerical_cols])

array([[-1.06119853e+00, -1.03182710e+00,  9.81338211e-01,
        -1.12861489e-01],
       [-1.31267830e+00,  1.29266307e+00,  1.10324526e+00,
        -1.12655904e-01],
       [ 9.86565309e-01, -7.50070712e-01,  6.15617059e-01,
        -1.05507587e-01],
       ...,
       [-2.55267319e-16, -1.63055941e+00,  2.09260225e-01,
        -1.10465017e-01],
       [ 1.59730189e+00,  4.47393917e-01,  8.85111558e+00,
        -1.05060505e-01],
       [ 9.14713946e-01, -1.06704664e+00, -8.06631862e-01,
        -1.12963389e-01]])

This is awesome, we dont need to concatenate the processing steps manually, but we still have a problem, we cant apply this numerical transformer to the whole dataframe

In [13]:
numerical_pipeline.fit_transform(data)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'mouse'

The pipeline fails when it tries to apply the imputer and the scaler to the non numerical columns.

We can fix this by applying a step that will select only the numerical columns before applying the imputer and the scaler. We can use the package named  [mlxtend](https://github.com/rasbt/mlxtend/pull/378) (which provides additional functionalities to scikit-learn) to import a ColumnSelector, which is a transformer that selects columns.

for example, we can do:

In [14]:
from mlxtend.feature_selection import ColumnSelector

numerical_col_selector = ColumnSelector(cols=numerical_cols)

We can apply this selector to the whole dataframe and it will automatically select the numerical columns

In [15]:
numerical_col_selector.fit_transform(data)

array([[ 1.90000000e+01,  2.10000000e+01,  7.60000000e+01,
         1.46815167e-01],
       [ 1.20000000e+01,  8.70000000e+01,  8.50000000e+01,
         2.10568899e-01],
       [ 7.60000000e+01,  2.90000000e+01,  4.90000000e+01,
         2.42733288e+00],
       ...,
       [            nan,  4.00000000e+00,  1.90000000e+01,
         8.89984688e-01],
       [ 9.30000000e+01,  6.30000000e+01,  6.57000000e+02,
         2.56597733e+00],
       [ 7.40000000e+01,  2.00000000e+01, -5.60000000e+01,
         1.15214954e-01]])

Now we can create a numerical pipeline that takes care of selection the appropriate columns

In [16]:
numerical_pipeline = make_pipeline(
    numerical_col_selector,
    imputer,
    scaler
)

And now we can apply this pipeline to the whole dataset

In [17]:
numerical_pipeline.fit_transform(data)[:5]

array([[-1.06119853, -1.0318271 ,  0.98133821, -0.11286149],
       [-1.3126783 ,  1.29266307,  1.10324526, -0.1126559 ],
       [ 0.98656531, -0.75007071,  0.61561706, -0.10550759],
       [-0.23490786,  0.55305256,  0.85943116, -0.10817076],
       [ 0.01657191, -0.25699704, -0.23773229, -0.04118222]])

 ### <font color='#B31B1B'> Text Pipeline </font>

There are a number of ways to process text features. One of the most popular is TF-IDF (Term frequency-inverse document frequency). The high level idea is that we'll create one feature per word in our dataset, and the value for this new feature for each row is going to be the multiplication of its 'term-frequency' (i.e. how often the word occurs in the text column for this row) by the word's inverse document frequency (i.e. 1/the fraction of rows that have that word). 

To do this in scikit-learn we need a [DenseTransformer](https://rasbt.github.io/mlxtend/user_guide/preprocessing/DenseTransformer/) that transforms the output produced by TfidfVectorizer (a sparse matrix) into a numpy array.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from mlxtend.preprocessing import DenseTransformer

text_pipeline = make_pipeline(
    ColumnSelector(cols=text_col, drop_axis=True),
    TfidfVectorizer(),
    DenseTransformer()
)

In [19]:
text_pipeline.fit_transform(data)

array([[0.        , 0.        , 0.10451175, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.27307095, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.27307095, 0.        ,
        0.        ],
       ...,
       [0.2728636 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.16929576,
        0.        ],
       [0.        , 0.13922624, 0.        , ..., 0.        , 0.        ,
        0.11417558]])

### <font color='#B31B1B'> Categorical Pipeline </font>
For categorical variables, we'll use **one-hot encoding**, which means we will create k binary columns (where k is the unique values for the variable). Then these columns will have all of their values 0 except one column that will represent the actual categorical value.

We can use the transformer `OneHotEncoder` that is part of the package [`category_encoders`](http://contrib.scikit-learn.org/categorical-encoding/), which is a package that adds additional categorical encoders that are compatible with scikit-learn.

In [21]:
from category_encoders import OneHotEncoder

Same way as before, we will do One-Hot Encoding

In [22]:
categorical_pipeline = make_pipeline(
     ColumnSelector(cols=categorical_col),
     OneHotEncoder()
)

categorical_pipeline.fit_transform(data)[:5]

,0_1,0_2,0_3,0_4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1


### <font color='#B31B1B'> Ordinal Pipeline </font>
To encode the ordinal column we need to first, use column selector to select it and then use OrdinalEncoder. We need to remember that ordinal Encoder requires the selection of the columns we want to transform, and because the output of ColumnSelector is a numpy array, we have to specify the column in position 0.

In [23]:
from category_encoders import OrdinalEncoder

# ColumnSelector's output is an array, so we use the column 0 for ordinal encoder
ordinal_encoder = OrdinalEncoder(mapping=[
    {"col": 0, 
      "mapping": {
        "very bad": 0,
        "bad": 1,
        "normal": 2,
        "good": 3,
        "very good": 4
      } 
     }
])
ordinal_pipeline = make_pipeline(
    ColumnSelector(cols=ordinal_col),
    ordinal_encoder
)

ordinal_pipeline.fit_transform(data)[:5]

,0
0,2
1,1
2,0
3,4
4,0


## <font color='#B31B1B'> Pipeline Union </font>

Now we have the individual pipelines that process each different datatype.

Now we just need to get a [FeatureUnion](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion) to join all of the outputs together. A feature union doesnt apply all of its steps sequentially. Instead, it applies the input to all of the steps at once and puts them together

![pipeline](images/pipeline.png)

In [ ]:
from sklearn.pipeline import make_union

In [ ]:
processing_pipeline = make_union(
    numerical_pipeline,
    text_pipeline,
    categorical_pipeline,
    ordinal_pipeline
)

In [ ]:
processing_pipeline

Now we can use this FeatureUnion to transform the whole dataset

In [ ]:
fit_data = processing_pipeline.fit_transform(data)

fit_data

Finally, we just have to add an estimator to the end of the pipeline, so it can be trained with the transformed data. Recall that the general syntax for training a model (outside of a pipeline) is fit then predict. For example, if we wanted to fit a linear regression on our pre-processed data we would:

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fit_data, data[target_variable])

#Now our model's fit and we can either pull out the coefficients or use it to make predictions!

We can embed this learning process into our pipeline

In [ ]:
estimator = LinearRegression()
estimator_pipeline = make_pipeline(
    processing_pipeline,
    estimator
)

Now that we added an estimator to the pipeline we can do `fit` and `predict`

In [ ]:
estimator_pipeline.fit(data.drop(target_variable, axis=1), data[target_variable])

In [ ]:
estimator_pipeline.predict(data)[:5]

Using pipelines not only helps organizing the different steps. We can also use all of scikit-learn tools with pipelines! For example, we can do cross validation with the estimator pipeline we just made.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
X = data.drop(columns=target_variable)
y = data[target_variable]

In [ ]:
mae_cv = cross_val_score(estimator_pipeline, 
                X, 
                y,
                scoring='neg_mean_absolute_error', 
                cv=5
)
mae_cv

In [ ]:
mae_cv.mean()

## <font color='#B31B1B'> Hyperparameter Optimization </font>

Because pipelines follow the scikit-learn convention of implementing the methods fit, transform and predict, we can use a search in a pipeline.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
predictive_pipeline = Pipeline(
    [
     ("processing", processing_pipeline),
     ("estimator", LinearRegression())   
    ])

This pipeline has a loot of hyperparams we can tweak, basically every single parameter on every single step of the pipeline, we  can see their names like this:

In [ ]:
sorted(predictive_pipeline.get_params().keys())

We'll focus on whether or not we want to fit an intercept in our linear regression model.

In [ ]:
param_dist_random = {
    "estimator__fit_intercept": [True, False],
}

To add hyperparameter tuning to our pipeline we simply set the estimator to be our predictive pipeline. There are a few ways to do automated hyperparameter selection in scikit-learn, RandomizedSearchCV and GridSearchCV. They both take a set of values to consider, but the first trys random selelctions of those hyperparameters, and the other looks at every possible combination. The second is thus guaranteed to do better but can take much longer (for those interested check out this <a href='https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf'> great paper </a> that compares the two.

We'll use randomized search, but the syntax is the same for grid search (except you have to put in a set of values, not a distribution of potential values).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_search_pipeline = RandomizedSearchCV(
    estimator=predictive_pipeline, 
    param_distributions=param_dist_random,
   scoring="neg_mean_squared_error", n_jobs=-1, n_iter=10)

Now we can perform the randomized grid search for the whole pipeline (under the hood, we're trying a bunch of different hyperparameters then doing CV to get an estimate of its performance, we then return the one with the best CV performance).

In [ ]:
%%time
random_search_pipeline.fit(X, y)

Let's take a peak at our best model + score.

In [ ]:
print(random_search_pipeline.best_score_)
print(random_search_pipeline.best_estimator_)

We can even see what final steps we've applied in our best model (this is super handy if we're trying out different hyperparameters during data pre-processing)

In [ ]:
random_search_pipeline.best_estimator_.steps[0]

## <font color='#B31B1B'> Exporting Pipelines </font>

The great thing about scikit learn pipelines is that we can save it, and reuse them **without having to retrain them**, we can use the library joblib to do so.

For example, to save the pipeline we have built:

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

In [ ]:
reloaded = load("pipeline.joblib")

Now we can predict directly!

In [ ]:
reloaded.predict(X)[:10]